# Pre-requisites:
Ensure that you have the following pre-requisites to run this notebook:

* Install two open source libraries in your terminal

Run 'pip install yahoo_fin'

Run 'pip install requests_html'

* Connection information to your (local) PostgreSQL database. You will need it for your config.py file that resides in the same folder as this notebook. Example:

API key for yahoofinanceapi web site

API_KEY = "######"

username = "######"

password = "######"

host = "localhost"

port = 5432

database = "######"


# 1. (EXTRACT) TRENDING STOCK SYMBOLS/TICKERS

API domain: https://yfapi.net

API path: /v1/finance/trending/{region}

Pull data (extract) from yahoofinancials API (URL path is /v1/finance/trending/{region}).
It returns stock symbols (up to 20) for regions around the world: US AU CA FR DE HK IT ES GB IN.
Pulling for each region.

In [1]:
# Import required modules and variables
import pandas as pd
import requests
import pprint
from config import API_KEY
from config import username, password, host, port, database

In [2]:
# Start with an empty dataframe having needed columns
finance_trends_df = pd.DataFrame({'symbol': [], 'region': [], 'startInterval': [], 'jobTimestamp': [] })
finance_trends_df


,symbol,region,startInterval,jobTimestamp


In [3]:
# Define connection variables
regions = ["US", "AU", "CA", "FR", "DE", "HK", "IT", "ES", "GB", "IN"]

url = "https://yfapi.net/v1/finance/trending/"

# Credentials to include
headers = {
    'x-api-key': API_KEY
    }

In [4]:
## Make API request for the number of regions of interest. Recursively save them
##  into the finance_trend_df dataframe by using 'append'
for region in regions:
    url_string = url + region
    response = requests.request("GET", url_string, headers=headers).json()
    response1_df = pd.DataFrame(response['finance']['result'][0]['quotes'])
    response1_df['region'] = region
    response1_df['startInterval'] = response['finance']['result'][0]['startInterval']
    response1_df['jobTimestamp'] = response['finance']['result'][0]['jobTimestamp']
    finance_trends_df = finance_trends_df.append(response1_df, ignore_index=True)

finance_trends_df.head()

,symbol,region,startInterval,jobTimestamp
0,NKE,US,2.021092e+11,1.632435e+12
1,COST,US,2.021092e+11,1.632435e+12
2,CRM,US,2.021092e+11,1.632435e+12
3,QS,US,2.021092e+11,1.632435e+12
4,EAR,US,2.021092e+11,1.632435e+12


In [5]:
## How many responses did we get?  10 regions
len(finance_trends_df['region'].unique())

10

# (TRANSFORMATION): change date/time on startInterval and jobTimestamp

In [6]:
from datetime import datetime

# Transform jobTimestamp into a date/time timestamp
finance_trends_df['jobTimestamp'] = pd.to_datetime(finance_trends_df['jobTimestamp'], unit='ms')

# Transform startInterval into a date/time format
finance_trends_df['startInterval'] = pd.to_datetime(finance_trends_df['startInterval'], format='%Y%m%d%H%M')


In [25]:
# Make the index be the ID Number column for the dataframe
# Reset index so that the index becomes the ID Number column
finance_trends_df = (finance_trends_df.reset_index()
                                      .rename(columns={"index": "id_number"}))

In [26]:
finance_trends_df.head()

,id_number,symbol,region,startInterval,jobTimestamp
0,0,NKE,US,2021-09-23 21:00:00,2021-09-23 22:09:33.356
1,1,COST,US,2021-09-23 21:00:00,2021-09-23 22:09:33.356
2,2,CRM,US,2021-09-23 21:00:00,2021-09-23 22:09:33.356
3,3,QS,US,2021-09-23 21:00:00,2021-09-23 22:09:33.356
4,4,EAR,US,2021-09-23 21:00:00,2021-09-23 22:09:33.356


In [7]:
# Loop through each stock symbol and make a list of the unique stock tickers
stock_list = [stock for stock in finance_trends_df['symbol'].unique()]
stock_list

['NKE',
 'COST',
 'CRM',
 'QS',
 'EAR',
 'PLTR',
 'AFRM',
 'NAK',
 'RNAZ',
 'DATS',
 'VTVT',
 'NVAX',
 'U',
 'ARBK',
 'AEHR',
 'BNTX',
 'CCL',
 'IHT',
 'GOEV',
 'JOBY',
 '^AORD',
 'BBD-B.TO',
 'TLRY',
 'AQN.TO',
 'FTS.TO',
 'TOU.TO',
 'BB.TO',
 'BABA',
 'CEI',
 'HIVE',
 'SAVA',
 'RY.TO',
 'TD.TO',
 '^FCHI',
 'AMC',
 '^HSI',
 'AAPL',
 '^DJI',
 'ENEL.MI',
 '^FTSE']

In [8]:
# How many unique stock tickers do we have?
len(stock_list)

40

,symbol,region,startInterval,jobTimestamp
0,NKE,US,2021-09-23 21:00:00,2021-09-23 22:09:33.356
1,COST,US,2021-09-23 21:00:00,2021-09-23 22:09:33.356
2,CRM,US,2021-09-23 21:00:00,2021-09-23 22:09:33.356
3,QS,US,2021-09-23 21:00:00,2021-09-23 22:09:33.356
4,EAR,US,2021-09-23 21:00:00,2021-09-23 22:09:33.356


# 2. (EXTRACT) REAL TIME QUOTE DATA

API domain: https://yfapi.net
API path: /v6/finance/quote
API variables:  querystring  {"symbols" : "string1,string2,string3,..."}

Pull data (extract) from yahoofinancials API (URL path is /v6/finance/quote) with "symbols" set to comma-separated list of up to 10 stock symbols at a time.

Use the query_string of 10 stock symbols.

In [10]:
# The quote API query wants the list of symbols as a single comma separated string 
#  with no spaces.  Max is 10 per request.
query_string=''
for stock in stock_list:
    query_string += f"{stock},"
query_string = query_string.rstrip(',')
query_string

'NKE,COST,CRM,QS,EAR,PLTR,AFRM,NAK,RNAZ,DATS,VTVT,NVAX,U,ARBK,AEHR,BNTX,CCL,IHT,GOEV,JOBY,^AORD,BBD-B.TO,TLRY,AQN.TO,FTS.TO,TOU.TO,BB.TO,BABA,CEI,HIVE,SAVA,RY.TO,TD.TO,^FCHI,AMC,^HSI,AAPL,^DJI,ENEL.MI,^FTSE'

In [11]:
# For now, get the first 10 and run the quote API
query_string=''
for i in range(10):
    query_string += f"{stock_list[i]},"
query_string = query_string.rstrip(',')
query_string

'NKE,COST,CRM,QS,EAR,PLTR,AFRM,NAK,RNAZ,DATS'

In [12]:
## Query of /v6/finance/quote API

url = "https://yfapi.net/v6/finance/quote"

## Warning: no space between symbols
querystring = {"symbols" : query_string}

# Credentials to include
headers = {
    'x-api-key': API_KEY
    }

In [13]:
## Make API request (query string required for this API)
response2 = requests.request("GET", url, headers=headers, params=querystring).json()

# (when troubleshooting) What does the response look like?
# pprint.pprint(response2['quoteResponse']['result'])

In [14]:
## How many records did we get?
len(response2['quoteResponse']['result'])

10

In [15]:
# Create a new DataFrame from all the results in the dictionary
finance_quote_df = pd.DataFrame(response2['quoteResponse']['result'])

In [16]:
# Get column names, pass to DBA for table creation
finance_quote_df.columns

Index(['language', 'region', 'quoteType', 'quoteSourceName', 'triggerable',
       'currency', 'exchange', 'longName', 'messageBoardId',
       'exchangeTimezoneName', 'exchangeTimezoneShortName',
       'gmtOffSetMilliseconds', 'market', 'esgPopulated', 'shortName',
       'fiftyTwoWeekRange', 'fiftyTwoWeekHighChange',
       'fiftyTwoWeekHighChangePercent', 'fiftyTwoWeekLow', 'fiftyTwoWeekHigh',
       'dividendDate', 'earningsTimestamp', 'earningsTimestampStart',
       'earningsTimestampEnd', 'trailingAnnualDividendRate', 'trailingPE',
       'trailingAnnualDividendYield', 'epsTrailingTwelveMonths', 'epsForward',
       'epsCurrentYear', 'priceEpsCurrentYear', 'sharesOutstanding',
       'bookValue', 'fiftyDayAverage', 'fiftyDayAverageChange',
       'fiftyDayAverageChangePercent', 'twoHundredDayAverage',
       'twoHundredDayAverageChange', 'twoHundredDayAverageChangePercent',
       'marketCap', 'forwardPE', 'priceToBook', 'sourceInterval',
       'exchangeDataDelayedBy', 'averag

In [17]:
finance_quote_df.head()

,language,region,quoteType,quoteSourceName,triggerable,currency,exchange,longName,messageBoardId,exchangeTimezoneName,...,financialCurrency,regularMarketOpen,averageDailyVolume3Month,averageDailyVolume10Day,fiftyTwoWeekLowChange,fiftyTwoWeekLowChangePercent,marketState,displayName,symbol,ipoExpectedDate
0,en-US,US,EQUITY,Nasdaq Real Time Price,True,USD,NYQ,"NIKE, Inc.",finmb_291981,America/New_York,...,USD,158.71,6211209,6715850,40.780000,0.343266,POST,NIKE,NKE,NaN
1,en-US,US,EQUITY,Nasdaq Real Time Price,True,USD,NMS,Costco Wholesale Corporation,finmb_92817,America/New_York,...,USD,452.30,1690384,1916712,145.780000,0.474853,POST,Costco Wholesale,COST,NaN
2,en-US,US,EQUITY,Nasdaq Real Time Price,True,USD,NYQ,"salesforce.com, inc.",finmb_122917,America/New_York,...,USD,270.00,5926456,5333925,76.349990,0.378889,POST,salesforce.com,CRM,NaN
3,en-US,US,EQUITY,Nasdaq Real Time Price,True,USD,NYQ,QuantumScape Corporation,finmb_204748122,America/New_York,...,USD,23.77,9090792,12736562,15.799999,1.404444,POST,QuantumScape,QS,NaN
4,en-US,US,EQUITY,Nasdaq Real Time Price,True,USD,NMS,"Eargo, Inc.",finmb_285793760,America/New_York,...,USD,8.37,543170,677612,0.530000,0.083728,POST,Eargo,EAR,2020-10-16


# 3. (EXTRACT) STOCK HISTORY

Open Source Library:  yahoo_fin
Usage: get_data(ticker, start_date = mm/dd/YYYY, end_date = None, index_as_date = True, interval = “1d”)
API variables:  ticker      a single ticker symbol or a list of ticker symbols
                start_date  first date in requested history, back to 1970
                end_date    last date in requested hostory, to the present
                index_as_date  should date column be used as the dataframe index?
                interval    data interval of 1d, 1w, 1m for daily, weekly, monthly

Pull data (extract) from yahoo_fin library with "ticker" set to a list of up to 10 stock symbols at a time.  Specify data interval and range of desired time span.

Returns one dataframe of data per ticker

In [18]:
## Query of yahoo_fin
from yahoo_fin.stock_info import get_data

my_tickers=stock_list
my_startdate="12/04/2009"
my_enddate="09/23/2021"
my_indexdate=True
my_interval="1wk"

In [19]:
## Make API request.  Save all results to a dictionary

stock_history = {}
for ticker in my_tickers:
    try:
        stock_history[ticker] = get_data(ticker,
                                         start_date=my_startdate,
                                         end_date=my_enddate,
                                         index_as_date=my_indexdate,
                                         interval=my_interval)
        print(f"Good response on ticker {ticker}")
    except:
        print(f"No data on ticker {ticker}...")

Good response on ticker NKE
Good response on ticker COST
Good response on ticker CRM
Good response on ticker QS
Good response on ticker EAR
Good response on ticker PLTR
Good response on ticker AFRM
Good response on ticker NAK
Good response on ticker RNAZ
Good response on ticker DATS
Good response on ticker VTVT
Good response on ticker NVAX
Good response on ticker U
Good response on ticker ARBK
Good response on ticker AEHR
Good response on ticker BNTX
Good response on ticker CCL
Good response on ticker IHT
Good response on ticker GOEV
Good response on ticker JOBY
Good response on ticker ^AORD
Good response on ticker BBD-B.TO
Good response on ticker TLRY
Good response on ticker AQN.TO
Good response on ticker FTS.TO
Good response on ticker TOU.TO
Good response on ticker BB.TO
Good response on ticker BABA
Good response on ticker CEI
Good response on ticker HIVE
Good response on ticker SAVA
Good response on ticker RY.TO
Good response on ticker TD.TO
Good response on ticker ^FCHI
Good respon

In [20]:
## How many records did we get?
len(stock_history)

40

In [21]:
# stock_history is a dictionary of dataframes.  Using the .keys you can see which
#  tickers actually returned some data
stock_history.keys()

dict_keys(['NKE', 'COST', 'CRM', 'QS', 'EAR', 'PLTR', 'AFRM', 'NAK', 'RNAZ', 'DATS', 'VTVT', 'NVAX', 'U', 'ARBK', 'AEHR', 'BNTX', 'CCL', 'IHT', 'GOEV', 'JOBY', '^AORD', 'BBD-B.TO', 'TLRY', 'AQN.TO', 'FTS.TO', 'TOU.TO', 'BB.TO', 'BABA', 'CEI', 'HIVE', 'SAVA', 'RY.TO', 'TD.TO', '^FCHI', 'AMC', '^HSI', 'AAPL', '^DJI', 'ENEL.MI', '^FTSE'])

In [22]:
# Concatenate all dataframes
# Start with an empty dataframe having the needed columns
stock_history_df = pd.DataFrame({'open': [], 'high': [], 'low': [], 'close': [], 'adjclose': [], 'volume': [], 'ticker': [] })

# Loop through the stock tickers that received a response and add them to
#   stock_history_df
for stock in stock_history.keys():
    stock_history_df = stock_history_df.append(stock_history[stock])
    
# Reset index so that date field is a named column
stock_history_df = (stock_history_df.reset_index()
                                    .rename(columns={"index": "date_db"}))
len(stock_history_df)

17740

In [23]:
stock_history_df.head()

,date_db,open,high,low,close,adjclose,volume,ticker
0,2009-11-30,16.315001,16.462500,15.9550,16.075001,13.879978,21214800.0,NKE
1,2009-12-07,16.075001,16.247499,15.4375,15.965000,13.842188,69729200.0,NKE
2,2009-12-14,16.002501,16.337500,15.7675,16.105000,13.963573,58504800.0,NKE
3,2009-12-21,16.129999,16.389999,16.0450,16.312500,14.143482,24088400.0,NKE
4,2009-12-28,16.312500,16.655001,16.1950,16.517500,14.321224,27010800.0,NKE


# 4. (TRANSFORMATION) other transformations and column reductions/renames

Transformations (either in the database or here in Jupyter Notebook):
Renaming columns, taking fewer columns into new dataframe before loading, calculating new needed columns, dropping duplicates

# 5. (LOAD) into database
Use the values pulled in from config.py to construct the database connection string.  Connect to the database.  Write the data frames to the database tables

In [28]:
from sqlalchemy import create_engine

# Define the engine string to be used for connecting to your database using the 
#  environment variables found in config.py
connection_string = f'{username}:{password}@{host}:{port}/{database}'
engine = create_engine(f'postgresql://{connection_string}')

In [29]:
# Write finance_trends to database
finance_trends_df.to_sql(name='finance_trends', con=engine, if_exists='append', index=False)

In [27]:
# Write finance_quote to database
finance_quote_df.to_sql(name='finance_quote', con=engine, if_exists='append', index=False)

In [28]:
# Write stock_history to database
stock_history_df.to_sql(name='stock_history', con=engine, if_exists='append', index=False)

In [30]:
# Verify that there is data in finance_trends
pd.read_sql_query('select * from finance_trends', con=engine).head()

,id_number,symbol,region,startInterval,jobTimestamp
0,0,NKE,US,2021-09-23,2021-09-23 22:09:33.356
1,1,COST,US,2021-09-23,2021-09-23 22:09:33.356
2,2,CRM,US,2021-09-23,2021-09-23 22:09:33.356
3,3,QS,US,2021-09-23,2021-09-23 22:09:33.356
4,4,EAR,US,2021-09-23,2021-09-23 22:09:33.356


In [30]:
# Verify that there is data in finance_quote
pd.read_sql_query('select * from finance_quote', con=engine).head()

,ask,askSize,averageAnalystRating,averageDailyVolume10Day,averageDailyVolume3Month,bid,bidSize,bookValue,currency,displayName,...,nameChangeDate,newListingDate,circulatingSupply,lastMarket,volume24Hr,volumeAllCurrencies,fromCurrency,toCurrency,startDate,coinImageUrl
0,10.80,9,3.9 - Underperform,8836012,10474489,10.79,9,2.571,USD,BlackBerry,...,None,None,None,None,None,None,None,None,None,None
1,348.45,14,1.9 - Buy,19128737,14037237,348.39,13,48.913,USD,Facebook,...,None,None,None,None,None,None,None,None,None,None
2,45.76,18,2.5 - Buy,6073312,23926582,45.71,10,-8.533,USD,Robinhood Markets,...,None,None,None,None,None,None,None,None,None,None
3,27.35,13,2.1 - Buy,16214775,19743384,27.36,8,8.462,USD,Plug Power,...,None,None,None,None,None,None,None,None,None,None
4,14.11,8,None,181525,43003,14.02,14,101.868,USD,MEDIROM Healthcare,...,None,None,None,None,None,None,None,None,None,None


In [31]:
# Verify that there is data in stock_history
pd.read_sql_query('select * from stock_history', con=engine).head()

,date_db,open,high,low,close,adjclose,volume,ticker,jobTimestamp
0,2009-11-30,59.799999,60.290001,58.130001,58.750000,58.750000,27307600.0,BB,None
1,2009-12-07,59.040001,66.500000,58.860001,63.840000,63.840000,99963600.0,BB,None
2,2009-12-14,64.059998,71.599998,62.529999,70.000000,70.000000,147390400.0,BB,None
3,2009-12-21,70.330002,71.540001,66.500000,66.919998,66.919998,62976600.0,BB,None
4,2009-12-28,67.410004,68.690002,66.500000,67.540001,67.540001,34025200.0,BB,None
